In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pickle

In [2]:
filename = 'nlp_model.pkl'
clf = pickle.load(open(filename, 'rb'))
vectorizer = pickle.load(open('tranform.pkl','rb'))

### Trial for one particular movie

In [3]:
imdbID = 'tt10888594'
url1 = 'https://www.imdb.com/title/'+imdbID+'/reviews?sort=helpfulnessScore&dir=desc&ratingFilter=0'
x = requests.get(url1)
soup = BeautifulSoup(x.text,"html.parser")


In [4]:
mov_cont = soup.find_all('div', class_ = 'text show-more__control')
print (type(mov_cont))
print (len(mov_cont))

<class 'bs4.element.ResultSet'>
25


In [5]:
reviews_list = [] # list of reviews
reviews_status = [] # list of comments (good or bad)
for reviews in mov_cont:
    if reviews.string:
        reviews_list.append(reviews.string)
        movie_review_list = np.array([reviews.string])
        movie_vector = vectorizer.transform(movie_review_list)
        pred = clf.predict(movie_vector)
        reviews_status.append('Good' if pred else 'Bad')

In [6]:
scrap = pd.DataFrame({'review':reviews_list[:8],'sentiment':reviews_status[:8]})
scrap.loc[:, 'review'] = scrap['review'].str.replace("\n","")
scrap

,review,sentiment
0,Salman Khan needs to take retirement. This mov...,Bad
1,Super duper flop. Old wine in new bottleIt's l...,Bad
2,Dont waste your time on this garbage Salman Kh...,Bad
3,I would rather prefer watching TVF webseries a...,Bad
4,Let me come straight to the point - this movie...,Bad
5,We don't eserve another pandemic . We understa...,Bad
6,Typical Salman khan film with no logic and no ...,Bad
7,I knew it. This movie is going to be trash whe...,Good


### Testing for movie from dataset

In [9]:
df = pd.read_csv("movies_small.csv")

In [15]:
def search(title):
    filtered = df[df["Title"] == title]["imdbID"]
    imdbID = pd.DataFrame(filtered).to_string(columns=None, col_space=None, header=False, index=False,)
    url1 = 'https://www.imdb.com/title/'+imdbID+'/reviews?sort=helpfulnessScore&dir=desc&ratingFilter=0'
    x = requests.get(url1)
    soup = BeautifulSoup(x.text,"html.parser")
    mov_cont = soup.find_all('div', class_ = 'text show-more__control')
    reviews_list = [] # list of reviews
    reviews_status = [] # list of comments (good or bad)
    for reviews in mov_cont:
        if reviews.string:
            reviews_list.append(reviews.string)
            movie_review_list = np.array([reviews.string])
            movie_vector = vectorizer.transform(movie_review_list)
            pred = clf.predict(movie_vector)
            reviews_status.append('Good' if pred else 'Bad')
    scrap = pd.DataFrame({'review':reviews_list[:8],'sentiment':reviews_status[:8]})
    scrap.loc[:, 'review'] = scrap['review'].str.replace("\n","")
    return scrap

In [17]:
search('Pulp Fiction')

,review,sentiment
0,"This is Tarantino's masterpiece, there's no ot...",Good
1,Before I saw this I assumed it was probably ov...,Good
2,Just the best movie... I can imagine my family...,Good
3,"Pulp Fiction is the most original, rule breaki...",Good
4,One of the best movies I have ever seen is a f...,Good
5,"To put this in context, I am 34 years old and ...",Good
6,This movie is such a masterpiece for many reas...,Good
7,Can't help but love it!🖤 bcoz it's good cool b...,Good
